In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
# Load Dataset
df = pd.read_csv("laptop_price.csv", encoding="latin1")
print(df.head())

   laptop_ID Company      Product   TypeName  Inches  \
0          1   Apple  MacBook Pro  Ultrabook    13.3   
1          2   Apple  Macbook Air  Ultrabook    13.3   
2          3      HP       250 G6   Notebook    15.6   
3          4   Apple  MacBook Pro  Ultrabook    15.4   
4          5   Apple  MacBook Pro  Ultrabook    13.3   

                     ScreenResolution                         Cpu   Ram  \
0  IPS Panel Retina Display 2560x1600        Intel Core i5 2.3GHz   8GB   
1                            1440x900        Intel Core i5 1.8GHz   8GB   
2                   Full HD 1920x1080  Intel Core i5 7200U 2.5GHz   8GB   
3  IPS Panel Retina Display 2880x1800        Intel Core i7 2.7GHz  16GB   
4  IPS Panel Retina Display 2560x1600        Intel Core i5 3.1GHz   8GB   

                Memory                           Gpu  OpSys  Weight  \
0            128GB SSD  Intel Iris Plus Graphics 640  macOS  1.37kg   
1  128GB Flash Storage        Intel HD Graphics 6000  macOS  1.34kg   

In [4]:
# Targetnya price
target_col = 'Price_euros'

# Binning targetnya biar jadi kategorikal
bins = [0, 30000, 50000, np.inf]
labels = ["Entry-level", "Mid-range", "High-end"]
df["Price_Class"] = pd.cut(df[target_col], bins=bins, labels=labels)

# Pisahkan fitur dan target
X = df.drop(columns=[target_col, "Price_Class"])
y = df["Price_Class"]

In [5]:
# Label Encoding untuk kolom kategorikal
for col in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Normalisasi fitur numerik
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode target
le_y = LabelEncoder()
y = le_y.fit_transform(y)  # Entry-level=0, Mid-range=1, High-end=2

In [6]:
# Split data ke data train dan data test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [16]:
# Buat tiga model sequantial dengan fungsi aktivasi ReLU dan softmax untuk output layernya
model = Sequential([
    Dense(128, activation='relu', input_shape=(X.shape[1],)),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax') # 3 Neuron untuk output layer karena ada 3 kelas
])

In [17]:
# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
# Training
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    verbose=1
)

Epoch 1/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 5.5980e-05 - val_accuracy: 1.0000 - val_loss: 4.2345e-05
Epoch 2/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 5.5355e-05 - val_accuracy: 1.0000 - val_loss: 3.9841e-05
Epoch 3/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 4.9345e-05 - val_accuracy: 1.0000 - val_loss: 3.7661e-05
Epoch 4/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 4.9733e-05 - val_accuracy: 1.0000 - val_loss: 3.5517e-05
Epoch 5/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 3.8168e-05 - val_accuracy: 1.0000 - val_loss: 3.3657e-05
Epoch 6/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 3.7990e-05 - val_accuracy: 1.0000 - val_loss: 3.1891e-05
Epoch 7/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 4.4492e-05 - val_accuracy: 1.0000 - val_loss: 3.0169e-05
Epoch 8/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 

In [13]:
# Evaluasi
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
test_loss, test_acc   = model.evaluate(X_test, y_test, verbose=0)

print(f"Akurasi Training : {train_acc*100:.2f}%")
print(f"Akurasi Validasi : {test_acc*100:.2f}%")

Akurasi Training : 100.00%
Akurasi Validasi : 100.00%
